In [ ]:
import findspark
findspark.init("/home/raj/spark/")


In [ ]:
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession

from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator

def GBT_regression():
    

    spark = SparkSession\
        .builder\
        .appName("Python Spark gbr ML example")\
        .getOrCreate()

# Load and parse the data file, converting it to a DataFrame.
    data = spark.read.format("csv")\
      .option("header", "true")\
      .option("inferSchema", "true")\
      .load("/home/raj/Downloads/notenook/adult2.csv")

    #data.show()


    categoricalColumns = ["workclass", "occupation"]

    stages = []
    for categoricalCol in categoricalColumns:
        stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol+"Index")
	#.fit(ad_data)
	#df_numeric = stringIndexer.transform(ad_data)
	#df_numeric.repartition(1).repartition(1).write.csv('indexer')
	#print df_numeric.select('workclass','workclassIndex').show(5)
	#In the above line for example, it takes workclass string and concatinates with the address("Index")
        encoder = OneHotEncoder(inputCol=categoricalCol+"Index", outputCol=categoricalCol+"classVec")
	#print encoder.outputCol
        stages += [stringIndexer, encoder]

    label_stringIdx = StringIndexer(inputCol = "income", outputCol = "label")

    stages += [label_stringIdx]

    numericCols = ["age", "hours_per_week"]

    assemblerInputs = map(lambda c: c + "classVec", categoricalColumns) 

    assemblerInputs

    assemblerInputs=list(assemblerInputs) + numericCols

    assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")

    stages += [assembler]

# Run the feature transformations.
#  - fit() computes feature statistics as needed.
#  - transform() actually transforms the features.

    pipeline = Pipeline(stages=stages)

    pipelineModel = pipeline.fit(data)
    dataset = pipelineModel.transform(data)

    dataset.printSchema()

    cols = data.columns

    selectedcols = ["label", "features"] + cols
    dataset = dataset.select(selectedcols)

    from pyspark.ml.feature import VectorIndexer

    dataset.printSchema()

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
    featureIndexer =VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(dataset)
    print(featureIndexer)



    featuredf=featureIndexer.transform(dataset)

    featuredf.show()

# Split the data into training and test sets (30% held out for testing)
    (trainingData, testData) = featuredf.randomSplit([0.7, 0.3], seed = 100)

    trainingData.show()

    testData.show()




# Train a GBT model.
    gbt = GBTRegressor(featuresCol="indexedFeatures", maxIter=10)

# Chain indexer and GBT in a Pipeline
    pipeline = Pipeline(stages=[featureIndexer, gbt])


# Train model.  This also runs the indexer.
    model = pipeline.fit(trainingData)

# Make predictions.
    predictions = model.transform(testData)

# Select example rows to display.
    predictions.select("prediction", "label", "features").show(5)

# Select (prediction, true label) and compute test error
    evaluator = RegressionEvaluator(
        labelCol="label", predictionCol="prediction", metricName="rmse")
    rmse = evaluator.evaluate(predictions)

    print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

    gbtModel = model.stages[1]

      # summary only

    return gbtModel


In [ ]:
GBT_regression()
